# **Exercise 11.1 Text Generation**



# **1. Import the libraries**

In [6]:
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
import tensorflow.keras.utils as ku 
from keras.preprocessing.text import Tokenizer
import pandas as pd
import numpy as np
from keras.callbacks import EarlyStopping
import string, os 
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)


# **2. Load the Dataset**

In [2]:
our_dir = '../../Chapter09/Datasets/'
our_headlines = []
for filename in os.listdir(our_dir):
    if 'Articles' in filename:
        article_df = pd.read_csv(our_dir + filename)
        our_headlines.extend(list(article_df.headline.values))
        break

our_headlines = [h for h in our_headlines if h != "Unknown"]
len(our_headlines)

831

# **3. Dataset Preperation**

## **3.1 Dataset Cleaning**

In [3]:
def clean_text(txt):
    txt = "".join(v for v in txt if v not in string.punctuation).lower()
    txt = txt.encode("utf8").decode("ascii",'ignore')
    return txt 

corpus = [clean_text(x) for x in our_headlines]
corpus[60:80]

['lets go for a win on opioids',
 'floridas vengeful governor',
 'how to end the politicization of the courts',
 'when dr king came out against vietnam',
 'britains trains dont run on time blame capitalism',
 'questions for no license plates here using art to transcend prison walls',
 'dry spell',
 'are there subjects that should be offlimits to artists or to certain artists in particular',
 'that is great television',
 'thinking in code',
 'how gorsuchs influence could be greater than his vote',
 'new york today how to ease a hangover',
 'trumps gifts to china',
 'at penn station rail mishap spurs large and lasting headache',
 'chemical attack on syrians ignites worlds outrage',
 'adventure is still on babbos menu',
 'swimming in the fast lane',
 'a national civics exam',
 'obama adviser is back in the political cross hairs',
 'the hippies have won']

## **3.2 Generating Sequence of N-gram Tokens**

In [4]:
tokenizer = Tokenizer()

def get_seq_of_tokens(corpus):
    ## tokenization
    tokenizer.fit_on_texts(corpus)
    all_words = len(tokenizer.word_index) + 1
    
    ## convert data to sequence of tokens 
    input_seq = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_seq.append(n_gram_sequence)
    return input_seq, all_words

our_sequences, all_words = get_seq_of_tokens(corpus)
our_sequences[:20]

[[169, 17],
 [169, 17, 665],
 [169, 17, 665, 367],
 [169, 17, 665, 367, 4],
 [169, 17, 665, 367, 4, 2],
 [169, 17, 665, 367, 4, 2, 666],
 [169, 17, 665, 367, 4, 2, 666, 170],
 [169, 17, 665, 367, 4, 2, 666, 170, 5],
 [169, 17, 665, 367, 4, 2, 666, 170, 5, 667],
 [6, 80],
 [6, 80, 1],
 [6, 80, 1, 668],
 [6, 80, 1, 668, 10],
 [6, 80, 1, 668, 10, 669],
 [670, 671],
 [670, 671, 129],
 [670, 671, 129, 672],
 [673, 674],
 [673, 674, 368],
 [673, 674, 368, 675]]

## **3.3 Padding the Sequences and obtain variables: Predictors and Target**

In [7]:
def generate_padded_sequences(input_seq):
    max_sequence_len = max([len(x) for x in input_seq])
    input_seq = np.array(pad_sequences(input_seq, maxlen=max_sequence_len, padding='pre'))
    
    predictors, label = input_seq[:,:-1],input_seq[:,-1]
    label = ku.to_categorical(label, num_classes=all_words)
    return predictors, label, max_sequence_len

predictors, label, max_sequence_len = generate_padded_sequences(our_sequences)

# **4. LSTM's for Text Generation**

In [8]:
def create_model(max_sequence_len, all_words):
    input_len = max_sequence_len - 1
    model = Sequential()
    
    # Add Input Embedding Layer
    model.add(Embedding(all_words, 10, input_length=input_len))
    
    # Add Hidden Layer 1 - LSTM Layer
    model.add(LSTM(100))
    model.add(Dropout(0.1))
    
    # Add Output Layer
    model.add(Dense(all_words, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam')
    
    return model

model = create_model(max_sequence_len, all_words)
model.summary()

2021-10-29 13:18:42.378743: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 18, 10)            24220     
_________________________________________________________________
lstm (LSTM)                  (None, 100)               44400     
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 2422)              244622    
Total params: 313,242
Trainable params: 313,242
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.fit(predictors, label, epochs=100, verbose=5)

2021-10-29 13:18:44.202152: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/100
Epoch 2/100
Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100
Epoch 22/100
Epoch 23/100
Epoch 24/100
Epoch 25/100
Epoch 26/100
Epoch 27/100
Epoch 28/100
Epoch 29/100
Epoch 30/100
Epoch 31/100
Epoch 32/100
Epoch 33/100
Epoch 34/100
Epoch 35/100
Epoch 36/100
Epoch 37/100
Epoch 38/100
Epoch 39/100
Epoch 40/100
Epoch 41/100
Epoch 42/100
Epoch 43/100
Epoch 44/100
Epoch 45/100
Epoch 46/100
Epoch 47/100
Epoch 48/100
Epoch 49/100
Epoch 50/100
Epoch 51/100
Epoch 52/100
Epoch 53/100
Epoch 54/100
Epoch 55/100
Epoch 56/100
Epoch 57/100
Epoch 58/100
Epoch 59/100
Epoch 60/100
Epoch 61/100
Epoch 62/100
Epoch 63/100
Epoch 64/100
Epoch 65/100
Epoch 66/100
Epoch 67/100
Epoch 68/100
Epoch 69/100
Epoch 70/100
Epoch 71/100
Epoch 72/100
Epoch 73/100
Epoch 74/100
Epoch 75/100
Epoch 76/100
Epoch 77/100
Epoch 78

# **5. Generating the Text**

In [10]:
def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict_classes(token_list, verbose=0)
        
        output_word = ""
        for word,index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " "+output_word
    return seed_text.title()

# **6. Results**

In [11]:
print (generate_text("10 Ways", 11, model, max_sequence_len))
print (generate_text("europe looks to", 8, model, max_sequence_len))
print (generate_text("best way", 10, model, max_sequence_len))
print (generate_text("homeless in", 10, model, max_sequence_len))
print (generate_text("Unexpected results", 10, model, max_sequence_len))
print (generate_text("critics warn", 10, model, max_sequence_len))

10 Ways To Teach And Learn About Poetry With The New York Times
Europe Looks To State Or Florence Hold Part On Its Chemical
Best Way To Stand A Small Equal Large And Lasting Headache Russian
Homeless In The Dreaded Trump Curse Partisan Tool Not The Old Days
Unexpected Results Trumps Fat History Lesson In Play Catch Thats Tricky Say
Critics Warn Leader Moves To Hold Early Elections Shakier Flowers 5 Home
